In [9]:
%matplotlib inline

import collections
import matplotlib.mlab as mlab
import csv
import pandas as pd
import numpy as np
import scipy as sp
from cvxpy import *
import matplotlib.pyplot as plt
import os
import copy
import sys
import mosek

In [10]:
class species(object):
    def __init__(self, Nres, strat_dist, lin, strat_p=0.1):
        if strat_dist == 'uniform':
            self.strategy = [np.random.uniform() for i in range(Nres)]
        elif strat_dist == 'binomial':
            self.strategy = np.random.choice([0, 1], size=(Nres), p=[1-strat_p, strat_p])
        else:
            print 'Incorrectly specified distribution: options include "uniform" and "binomial".'
        if np.sum(self.strategy)!= 0:
            self.strat_norm = self.strategy/np.float64(np.sum(self.strategy))
        else:
            self.strat_norm = np.float64(self.strategy)
        self.fitness = np.exp(np.random.normal(0, 0.0001))
        self.fit_norm = self.fitness
        self.cost = 1
        self.lineage = lin
        self.barcode = [lin]
        self.frequency = []
    
    def perturb_fitness(self, epsilon):
        self.fitness = self.fitness*np.exp(np.random.normal(0, epsilon))
        #self.fit_norm = self.fitness
        return
    
    def normalize_fitness(self, av_fit):
        self.fit_norm = self.fitness/av_fit
        return
    
    def perturb_strat_rand(self, n=1):
        idx = np.random.choice(range(len(self.strategy)))
        self.strategy[idx] = 1-self.strategy[idx]
        if np.sum(self.strategy)!= 0:
            self.strat_norm = self.strategy/np.float64(np.sum(self.strategy))
        else:
            self.strat_norm = np.float64(self.strategy)
        return
    
    def perturb_strat_dir(self, idx):
        self.strategy[idx] = 1-self.strategy[idx]
        if np.sum(self.strategy)!= 0:
            self.strat_norm = self.strategy/np.float64(np.sum(self.strategy))
        else:
            self.strat_norm = np.float64(self.strategy)
        return
    
    def perturb_strategy_cont(self, epsilon):
        self.strategy = self.strategy+[epsilon*np.random.normal() for i in range(Nres)]
        self.strategy = self.strategy/np.sum(self.strategy)
        return
    
    def rebarcode(self, new_barcode):
        self.barcode.append(new_barcode)
        return
    
    def refrequency(self, frequency):
        self.frequency.append(frequency)
        return

In [4]:
def row_sparsify(A):
    asub = []
    aspec = []
    for i in range(len(A)):
        asub.append([])
        aspec.append([])
        for j in range(len(A[i])):        
            if A[i][j] > 0:
                asub[i].append(j)
                aspec[i].append(A[i][j])
    return asub, aspec

def run_opt(vspecies, vres):
    Nspec = len(vspecies)
    Nres = len(vres)
    
    with mosek.Env() as env:
        with env.Task(0,0) as task:
            
            task.putdouparam(mosek.dparam.intpnt_nl_tol_dfeas,0.0)
            task.putdouparam(mosek.dparam.intpnt_nl_tol_rel_gap,1e-14)
            task.putdouparam(mosek.dparam.intpnt_nl_tol_pfeas,0.0)
            
            numvar = Nres
            numcon = Nspec
            
            inf = 0.0

            bkx = [ mosek.boundkey.lo ] * numvar
            blx = [ 0.0 ] * numvar
            bux = [ inf ] * numvar
            
            bkc = [ mosek.boundkey.up ] * numcon
            buc = [1.0/vspecies[i].fit_norm for i in range(Nspec)]

            task.appendvars(numvar)
            task.appendcons(numcon)

            task.putvarboundslice(0, numvar, bkx, blx, bux)
            
            A = [vspecies[i].strat_norm for i in range(Nspec)]
            asub, aval = row_sparsify(A)
            
            for i in range(numcon):
                task.putbound(mosek.accmode.con,i,bkc[i],-inf,buc[i])
                task.putarow(i, asub[i], aval[i]);

            opro  = [mosek.scopr.log for i in range(Nres)]
            oprjo = range(Nres)
            oprfo = -vres
            oprgo = [1.0 for i in range(Nres)]
            oprho = [0.0 for i in range(Nres)]

            task.putSCeval(opro, oprjo, oprfo, oprgo, oprho)

            task.optimize()

            res = [ 0.0 ] * numvar
            task.getsolutionslice(
                mosek.soltype.itr,
                mosek.solitem.xx,
                0, numvar,
                res)
            
            delta = np.array(A).dot(np.array(res))-np.array(buc)
                
            return res, delta

In [26]:
# Run random environment with random initial species distribution
def run_rand_env(Nres, alpha, eps):
    Nspec = int(alpha*Nres)
    
    #Normalize influx for rescaled equations
    vres = np.ones(Nres)+np.random.normal(0, eps, size=Nres)
    vres = vres/sum(vres)
    vspecies = [species(Nres, 'binomial', i) for i in range(Nspec)]
    h, delta = run_opt(vspecies, vres)

    # Numerical precision is 1e-6, so for |delta|<1e-6,
    # set to zero.
    present = []
    absent = []
    
    for i in range(Nspec):
        if np.abs(delta[i])<0.000001:
            present.append(vspecies[i])
        else:
            absent.append(vspecies[i])
    return vres, h, present, absent

# Run specified environment
def run_spec_env(vres, vspecies):
    Nspec = len(vspecies)
    
    #run optimization scheme
    if Nspec == 1:
        h = [vres[i]/vspecies[0].strat_norm[i] for i in range(len(vres))]
        h = h/vspecies[0].fit_norm
        delta = [0]
    else:
        h, delta = run_opt(vspecies, vres)
    
    #presence/absence vector
    present = []
    absent = []
    
    # Numerical precision is 1e-6, so for |delta|<1e-6,
    # set to zero.
    for i in range(Nspec):
        if np.abs(delta[i])>1e-8:
            absent.append(vspecies[i])
        else:
            present.append(vspecies[i])
    
    return vres, h, present, absent

def calculate_frequencies(present, vres, h):
    a = np.array([present[i].fit_norm*present[i].strat_norm for i in range(len(present))])
    a = np.transpose(a)
    b = np.array([vres[i]/h[i] for i in range(len(vres))])
    m = sp.optimize.nnls(a, b)[0]
    return m

def calc_inv_fit(s, h):
    return s.fit_norm*np.dot(s.strat_norm, h)-1

def pick_species(rates, spec_pool):
    norm = sum(rates)
    if norm > 0:
        temp = np.random.choice(spec_pool, p=rates/norm)
    else:
        temp = np.random.choice(spec_pool)
    return temp

def check_nonneg(rate):
    if rate < 0:
        return 0
    else:
        return rate
    
def create_pool(vspecies, vres, h, freq):
    pool = []
    inv_prob = []
    for i in range(len(vspecies)):
        for j in range(len(vres)):
            temp = copy.deepcopy(vspecies[i])
            temp.perturb_strat_dir(j)
            pool.append(temp)
            inv_prob.append(check_nonneg(calc_inv_fit(temp,h))*freq[i])
    return pool, inv_prob

def sswm_evol(vspecies, vres, h, s, strat_prob, fit_prob):
    
    #calculate frequencies
    freq = calculate_frequencies(vspecies, vres, h)
    
    fitcoin = np.random.uniform()
    stratcoin = np.random.uniform()
    
    #both fitness and strategy mutations
    if fitcoin <= fit_prob and stratcoin <= strat_prob:
        pool, inv_prob = create_pool(vspecies, vres, h, freq)
        temp = pick_species(inv_prob, pool)
        temp.perturb_fitness(s)
        vspecies.append(temp)
        av_fit = 0
        for s in vspecies:
            av_fit=av_fit+s.fitness
        av_fit = av_fit/float(len(vspecies))
        for s in vspecies:
            s.normalize_fitness(av_fit)
    
    #only fitness mutation
    elif fitcoin <= fit_prob:
        temp = pick_species(freq, vspecies)
        temp.perturb_fitness(s)
        vspecies.append(temp)
        av_fit = 0
        for s in vspecies:
            av_fit=av_fit+s.fitness
        av_fit = av_fit/float(len(vspecies))
        for s in vspecies:
            s.normalize_fitness(av_fit)
    
    #only strategy mutation
    elif stratcoin <= strat_prob:
        pool, inv_prob = create_pool(vspecies, vres, h, freq)
        temp = pick_species(inv_prob, pool)
        vspecies.append(temp)
    
    return vspecies

def runT(Nres, alpha, T, s, eps, ratio):
    
    #initialize environment
    vres, h, present, absent = run_rand_env(Nres, alpha, eps)
    
    #calculate mutation type probabilities
    if ratio >= 1:
        strat_prob = 1.0/ratio
        fit_prob = 1.0
    else:
        strat_prob = 1.0
        fit_prob = ratio
        
    #run timecourse
    for i in range(T):
        present = sswm_evol(present, vres, h, s, strat_prob, fit_prob)
        vres, h, present, absent = run_spec_env(vres, present)
    
    #return final population
    return vres, h, present

In [27]:
Nres=50
alpha = 4
T = 5000
ratio = [0.01, 0.1, 1, 10, 100]
eps = 1e-2
s = 1e-2
output3 = []


for r in ratio:
    temp = []
    for i in range(10):
        temp.append(runT(Nres, alpha, T, s, eps, r))
    output3.append(temp)

print output3

[[(array([ 0.02024386,  0.01995016,  0.0197452 ,  0.01957998,  0.02017517,
        0.02004829,  0.01954534,  0.01980502,  0.01981137,  0.02007215,
        0.02019663,  0.01991917,  0.01971593,  0.01988727,  0.02050193,
        0.02002686,  0.02001262,  0.02001066,  0.02008927,  0.02009805,
        0.02010907,  0.02008156,  0.01994879,  0.02014036,  0.0204848 ,
        0.02007406,  0.02005362,  0.02007921,  0.01955406,  0.02009671,
        0.01997786,  0.01991371,  0.02033883,  0.01975654,  0.02052281,
        0.01981374,  0.01993836,  0.0201098 ,  0.01963626,  0.01997852,
        0.0199327 ,  0.01979903,  0.01974105,  0.019887  ,  0.02005254,
        0.01989256,  0.02016132,  0.02009653,  0.02025056,  0.02014309]), [1.0302690752732286, 0.90072669811878359, 0.89681838599899533, 1.005701631324851, 1.0387514189332805, 0.99474861367569634, 1.1483186669575407, 0.85959308288724046, 1.0087131796968016, 0.90644886205103714, 1.0390277845266667, 1.0116965604005019, 1.0428142032674059, 1.02818058

In [8]:
print 1e-2

0.01
